## Adjusting Lexicons

### EmoLex - Italian

In [25]:
import re
import pandas as pd

f = open("../Data/Italian-NRC-EmoLex.txt", "r")

lines = f.readlines()

f.close()

emolex = []

for line in lines[1:]:
    temp = re.sub("\n", "", line)
    results = re.split("\t", temp)
    if len(re.split(", ", results[-1])) > 1:
        words = re.split(", ", results[-1])
        for word in words:
            emolex.append(results[:-1]+[word])
    else:
        emolex.append(results)

emolex = pd.DataFrame(emolex, columns=["English word", "Anger", "Anticipation", "Disgust", "Fear", "Joy", "Negative", "Positive", "Sadness", "Surprise", "Trust", "word"])
emolex = emolex.loc[: , ["word", "Anger", "Anticipation", "Disgust", "Fear", "Joy", "Negative", "Positive", "Sadness", "Surprise", "Trust"]]
emolex.to_csv("../Data/IT_emolex.csv", index=False)

emolex.head()

,word,Anger,Anticipation,Disgust,Fear,Joy,Negative,Positive,Sadness,Surprise,Trust
0,sconcertato,0,0,0,0,0,0,0,0,0,0
1,abaco,0,0,0,0,0,0,0,0,0,1
2,abbandono,0,0,0,1,0,1,0,1,0,0
3,abbandonato,1,0,0,1,0,1,0,1,0,0
4,abbandono,1,0,0,1,0,1,0,1,1,0


In [69]:
emolex = pd.read_csv("../Data/IT_emolex.csv")
ItEM = pd.read_csv("../Data/IT_emotion_lexicon_cosine.csv")
ItEM_norm = pd.read_csv("../Data/IT_emotion_lexicon_normalised_new.csv")
print(len(emolex),len(ItEM), len(ItEM_norm))

14157 29999 29999


### Sentiment Lexicon - DPLp-IT

In [4]:
import re
import pandas as pd

f = open("../Data/DPLp-IT_lrec2016.txt", "r")

lines = f.readlines()

f.close()

sentiment_lexicon = []

for line in lines:

    temp = re.sub("\n", "", line)
    results = re.split("::|\t|,|\n", temp)
    sentiment_lexicon.append(results)

sentiment_lexicon = pd.DataFrame(sentiment_lexicon, columns=["lemma", "pos", "pos_score", "neg_score", "neu_score"])
sentiment_lexicon.to_csv("../Data/IT_sentiment_lexicon.csv", index=False)

sentiment_lexicon.head()

,lemma,pos,pos_score,neg_score,neu_score
0,essere,v,0.3675422,0.4671061,0.16535169
1,avere,v,0.27894887,0.60701084,0.1140403
2,fare,v,0.4421229,0.40804362,0.14983346
3,stare,v,0.31763914,0.4706841,0.21167673
4,dire,v,0.44390386,0.3657567,0.1903395


In [6]:
print(len(sentiment_lexicon.lemma.unique()))
print(len(sentiment_lexicon))
print(sentiment_lexicon.pos.unique())

# ['v' 's' 'a' 'b' 'h']: v: verb, s: noun, a: adjective, b: adverb, h: hashtag 

65273
75021
['v' 's' 'a' 'b' 'h']


In [7]:
import pandas as pd

sent_lexicon = pd.read_csv("../Data/IT_sentiment_lexicon.csv")
len(sent_lexicon)

75021

### Emotion Lexicon - ITeM

In [39]:
import pandas as pd

emotion = pd.read_csv("../Data/ItEM.FBNEWS15.cos", sep="\t")
print(len(emotion))
emotion.head()

239946


,emotion,word,cosine
0,gioia,festoso-a,0.647874
1,gioia,euforico-a,0.622582
2,gioia,esilarante-a,0.622579
3,gioia,gaio-a,0.617334
4,gioia,divertito-a,0.614806


In [127]:
import numpy as np
from tqdm.notebook import tqdm


ItEM = pd.read_csv("../Data/IT_emotion_lexicon_cosine.csv")
ItEM = ItEM.round(2)
results = {col_name: [] for col_name in ItEM.columns}
for _, row in tqdm(ItEM.iterrows(), total=len(ItEM)):
    d = row.drop(["word", "pos"]).to_dict()
    bin_vals = {key:int(d[key]==max(d.values())) for key in d.keys()}
    temp = {"word":row["word"], "pos":row["pos"]}
    temp |= bin_vals
    for key in temp.keys():
        results[key].append(temp[key])

ItEM_bin_df = pd.DataFrame.from_dict(results)
ItEM_bin_df.to_csv("../Data/IT_emotion_lexicon_binary_rounded.csv", index=False)
ItEM_bin_df.head()

  0%|          | 0/29999 [00:00<?, ?it/s]

,word,pos,joy,anger,surprise,disgust,fear,sadness,trust,anticipation
0,festoso,a,1,0,0,0,0,0,0,0
1,euforico,a,1,0,0,0,0,0,0,0
2,esilarante,a,1,0,0,0,0,0,0,0
3,gaio,a,1,0,0,0,0,0,0,0
4,divertito,a,1,0,0,0,0,0,0,0


In [40]:
# test_word = emotion.word.unique().tolist()[0]
# print(test_word.split("-"))
# print(emotion.loc[(emotion["word"]==test_word) & (emotion["emotion"]=="rabbia")]["cosine"].tolist()[0])

from tqdm.notebook import tqdm
import spacy

nlp = spacy.load("it_core_news_sm")

ENG_IT_emotions = {
    "joy": "gioia",
    "anger": "rabbia",
    "surprise": "sorpresa",
    "disgust": "disgusto",
    "fear": "paura",
    "sadness": "tristezza",
    "trust": "fiducia",
    "anticipation": "attese"
}

ENG_emotions = ["joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"]

results = []
for word in tqdm(emotion["word"].unique()):
    actual_word, pos = word.split("-")
    lemma = nlp(actual_word)[0].lemma_.lower()
    IT_emotions = emotion.loc[emotion["word"]==word]["emotion"].tolist()
    temp = []
    norm_factor = 0
    for eng_emo in ENG_emotions:
        emo_cosine = emotion.loc[(emotion["word"]==word) & (emotion["emotion"]==ENG_IT_emotions[eng_emo])]["cosine"].tolist()
        if len(emo_cosine) == 0:
            temp.append(0)
            norm_factor += 0
        else:
            temp.append(emo_cosine[0])
            norm_factor += emo_cosine[0]
    
    temp_new = [t/norm_factor for t in temp]
    emo_norm_score = [actual_word, lemma, pos] + temp_new
    results.append(emo_norm_score)

emotion_cosine = pd.DataFrame(results, columns=["actual_word", "lemma", "pos", "joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"])
emotion_cosine.to_csv("../Data/IT_emotion_lexicon_normalised_new.csv", index=False)
emotion_cosine.head()

  0%|          | 0/29999 [00:00<?, ?it/s]

,actual_word,lemma,pos,joy,anger,surprise,disgust,fear,sadness,trust,anticipation
0,festoso,festoso,a,0.288792,0.092691,0.103564,0.089147,0.098167,0.148815,0.095408,0.083415
1,euforico,euforico,a,0.211809,0.106492,0.129965,0.100949,0.113435,0.135096,0.105778,0.096476
2,esilarante,esilarare,a,0.246015,0.094562,0.094376,0.128638,0.112556,0.137287,0.093584,0.092982
3,gaio,gaio,a,0.248166,0.097285,0.085680,0.096044,0.118533,0.129797,0.133886,0.090609
4,divertito,divertire,a,0.194127,0.114506,0.142627,0.103522,0.108127,0.137351,0.104210,0.095530


In [41]:
emotion_cosine.pos.unique()

# ['a', 'v', 's'], a: adjective, v: verb, s: noun

array(['a', 'v', 's'], dtype=object)

In [42]:
len(emotion_cosine)

29999

In [44]:
sum(emotion_cosine.drop(columns=["actual_word","lemma", "pos"]).iloc[50])

1.0

## Computing sentiment and emotion vectors

### Lexicon-based approach: LexIT

In [1]:
import spacy
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords_list = stopwords.words('italian')

nlp = spacy.load("it_core_news_sm")
emotion_names = ["joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"]  
emotion_lexicon = pd.read_csv("../Data/IT_emotion_lexicon_binary.csv")
sentiment_lexicon = pd.read_csv("../Data/IT_sentiment_lexicon.csv")

def calc_emotion_profile(text):

    scores = {
        "positive": [],
        "negative": [],
        "neutral": [],
        "joy": [],
        "anger": [],
        "surprise": [],
        "disgust": [],
        "fear": [],
        "sadness": [],
        "trust": [],
        "anticipation": []
    }

    doc = nlp(text)
    for token in doc:
        # if token.is_alpha == True and token.text.lower() not in stopwords_list:
        if token.is_alpha == True:
            word = token.text.lower()
            lemma = token.lemma_.lower()
            pos = token.tag_[0].lower()
            sentiments = sentiment_lexicon.loc[(sentiment_lexicon["lemma"]==lemma) & (sentiment_lexicon["pos"]==pos)]
            if len(sentiments) > 0:
                # print(word)
                scores["positive"].append(sentiments["pos_score"].values[0])
                scores["negative"].append(sentiments["neg_score"].values[0])
                scores["neutral"].append(sentiments["neu_score"].values[0])
            else:
                scores["positive"].append(0)
                scores["negative"].append(0)
                scores["neutral"].append(1)
            
            emotions = emotion_lexicon.loc[(emotion_lexicon["word"]==word) & (emotion_lexicon["pos"]==pos)]
            if len(emotions) > 0:
                # print(word)
                for emo in emotion_names:
                    scores[emo].append(emotions[emo].values[0])

    emo_profile = {}
    norm_factor = 0
    for key in scores.keys():
        if len(scores[key]) > 0:
            emo_val = np.average(scores[key])*100
        else:
            emo_val = 0.00
        emo_profile[key] = emo_val
        if key in emotion_names:
            norm_factor += emo_val

    if norm_factor > 0:
        for key in emotion_names:
            emo_profile[key] = emo_profile[key]*100/norm_factor

    return emo_profile

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hrishitachakra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
## sanity check of EP computation

text_examples = [
    "sono felice",
    "La deforestazione è un male",
    "Questo è spaventoso",
    "Questa è una mela marcia"
]

results = {
    "text":[],
    "positive":[],
    "negative": [],
    "neutral": [],
    "sent_sum": [],
    "anger":[],
    "anticipation":[],
    "disgust": [],
    "fear":[],
    "joy": [],
    "sadness": [],
    "surprise": [],
    "trust": [],
    "emo_sum": []
}

for text in text_examples:
    EP_score = calc_emotion_profile(text)
    results["text"].append(text)
    for key in EP_score.keys():
        results[key].append(EP_score[key])
    results["sent_sum"].append(EP_score["positive"]+EP_score["negative"]+EP_score["neutral"])
    results["emo_sum"].append(sum([EP_score[key] for key in EP_score.keys() if key not in ["text", "positive", "negative", "neutral"]]))


results_df = pd.DataFrame.from_dict(results)
results_df.drop(["positive", "negative", "neutral", "sent_sum"], axis=1)

,text,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,emo_sum
0,sono felice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0
1,La deforestazione è un male,0.0,0.0,0.0,50.0,0.0,50.0,0.0,0.0,100.0
2,Questo è spaventoso,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0
3,Questa è una mela marcia,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0


In [ ]:
# trial run

import pandas as pd

gemma = pd.read_csv("../Data/gemma_resp.csv")

test = gemma.Resp[0]
emo_vals = calc_emotion_profile(test)
print(emo_vals)

{'positive': np.float64(24.469596846153845), 'negative': np.float64(23.87863453846154), 'neutral': np.float64(51.65176876923078), 'joy': np.float64(0.0), 'anger': np.float64(0.0), 'surprise': np.float64(16.66666666666667), 'disgust': np.float64(0.0), 'fear': np.float64(66.66666666666669), 'sadness': np.float64(0.0), 'trust': np.float64(16.66666666666667), 'anticipation': np.float64(0.0)}


In [ ]:
# compiling Inside Out responses

import pandas as pd

QL_IT = pd.read_csv("../Data/InsideOutData/QL-IT.csv")
QL_IT["QID"] = ["qGEN"+str(i+1) for i in range(len(QL_IT))]
QL_IT["Prompt Type"] = ["General"] * len(QL_IT)
QL_IT = QL_IT.loc[QL_IT["position"]<6]

QL_IT_Em = pd.read_csv("../Data/InsideOutData/QL-IT-Em.csv")
QL_IT_Em["QID"] = ["qEM"+str(i+1) for i in range(len(QL_IT_Em))]
QL_IT_Em["Prompt Type"] = ["Emotionally Charged"] * len(QL_IT_Em)
QL_IT_Em = QL_IT_Em.loc[QL_IT_Em["position"]<6]

IO_emotional_profile = {
    "QID":[],
    "rank": [],
    "IAS": [],
    "positive":[],
    "negative": [],
    "neutral": [],
    "anger":[],
    "anticipation":[],
    "disgust": [],
    "fear":[],
    "joy": [],
    "sadness": [],
    "surprise": [],
    "trust": [],
}

InsideOut_raw = pd.concat([QL_IT, QL_IT_Em], ignore_index=True)
InsideOut_raw["IAS"] = ["Bing"] * len(InsideOut_raw)
InsideOut_raw["date_generated"] = ["2020"] * len(InsideOut_raw)

InsideOut = InsideOut_raw[["QID","user_query", "Prompt Type", "target", "IAS", "TextSample", "position", "date_generated"]]
InsideOut.columns = ["QID","Query","Prompt Type","Task Sentiment", "IAS","Resp","Rank","date_generated"]
InsideOut.to_csv("../Data/InsideOutData_compiled.csv", index=False)
InsideOut.head(10)

,QID,Query,Prompt Type,Task Sentiment,IAS,Resp,Rank,date_generated
0,qGEN1,tornado,General,NaN,Bing,Tornado - Wikipedia Tornado - Wikipedia Tornad...,1,2020
1,qGEN2,tornado,General,NaN,Bing,"Extreme up-close video of tornado near Wray, C...",2,2020
2,qGEN3,tornado,General,NaN,Bing,Tornado facts and information. Tornadoes are v...,3,2020
3,qGEN4,tornado,General,NaN,Bing,Tornado Central - weather.com. Tornado Season ...,4,2020
4,qGEN5,tornado,General,NaN,Bing,Tornadoes | Ready.gov. Tornadoes can destroy b...,5,2020
5,qGEN11,tornado,General,NaN,Bing,Tornado - Wikipedia Tornado - Wikipedia Tornad...,1,2020
6,qGEN12,tornado,General,NaN,Bing,Tornado facts and information. Tornadoes are v...,2,2020
7,qGEN13,tornado,General,NaN,Bing,"Extreme up-close video of tornado near Wray, C...",3,2020
8,qGEN14,tornado,General,NaN,Bing,Tornado Central - weather.com. Tornado Season ...,4,2020
9,qGEN15,tornado,General,NaN,Bing,Tornadoes | Ready.gov. Tornadoes can destroy b...,5,2020


In [ ]:
# computing Inside Out EP

InsideOut = pd.read_csv("../Data/InsideOutData_compiled.csv")
IO_emotion_profile = {
    "QID":[],
    "Task Sentiment": [],
    "rank": [],
    "IAS": [],
    "positive":[],
    "negative": [],
    "neutral": [],
    "anger":[],
    "anticipation":[],
    "disgust": [],
    "fear":[],
    "joy": [],
    "sadness": [],
    "surprise": [],
    "trust": [],
}

for _, row in tqdm(InsideOut.iterrows(), total=len(InsideOut)):
    IO_emotion_profile["QID"].append(row["QID"])
    IO_emotion_profile["Task Sentiment"].append(row["Task Sentiment"])
    resp = row["Resp"]
    IO_emotion_profile["rank"].append(row["Rank"])
    IO_emotion_profile["IAS"].append("Bing")
    emo_profile = calc_emotion_profile(resp)
    for key in emo_profile.keys():
        IO_emotion_profile[key].append(emo_profile[key])

IO_EP_df = pd.DataFrame.from_dict(IO_emotion_profile)
IO_EP_df.to_csv("../Results/LexIT_queryWise_InsideOutEP.csv", index=False)
IO_EP_df.head()

  0%|          | 0/1739 [00:00<?, ?it/s]

,QID,Task Sentiment,rank,IAS,positive,negative,neutral,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,qGEN1,NaN,1,Bing,9.720957,9.432725,80.846318,12.5,12.5,0.0,18.75,18.75,0.000000,31.250000,6.25
1,qGEN2,NaN,2,Bing,17.297959,18.316977,64.385064,0.0,12.5,0.0,25.00,12.50,0.000000,50.000000,0.00
2,qGEN3,NaN,3,Bing,13.928572,13.945203,72.126225,0.0,0.0,0.0,20.00,40.00,0.000000,40.000000,0.00
3,qGEN4,NaN,4,Bing,18.422319,24.500678,57.077002,0.0,0.0,0.0,20.00,20.00,13.333333,46.666667,0.00
4,qGEN5,NaN,5,Bing,11.141217,13.802887,75.055896,0.0,20.0,0.0,0.00,60.00,0.000000,20.000000,0.00


In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

query_file_name = ""
query_file_path = "../Data/"+query_file_name
queries = pd.read_csv(query_file_path)
query_emotional_profile = {
    "QID":[],
    "positive":[],
    "negative": [],
    "neutral": [],
    "anger":[],
    "anticipation":[],
    "disgust": [],
    "fear":[],
    "joy": [],
    "sadness": [],
    "surprise": [],
    "trust": [],
}

for _, row in tqdm(queries.iterrows(), total = len(queries)):
    query_emotional_profile["QID"].append(row["QID"])
    query = row["Query"]
    # prompt_type = row["Prompt Type"]
    emo_profile = calc_emotion_profile(query)
    for key in emo_profile:
        query_emotional_profile[key].append(emo_profile[key])

query_EP_df = pd.DataFrame.from_dict(query_emotional_profile)
query_EP_df.to_csv("../Results/LexIT_queryWise_queryEP.csv", index=False)
query_EP_df.head()

  0%|          | 0/293 [00:00<?, ?it/s]

,QID,positive,negative,neutral,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,qGEN1,20.698589,35.330144,43.971260,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
1,qGEN2,20.698589,35.330144,43.971260,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
2,qGEN3,11.697892,9.843758,78.458350,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0
3,qGEN4,23.395784,19.687515,56.916700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,qGEN5,10.828426,9.374532,79.797041,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
bing = pd.read_csv("../Data/bing_resp.csv")
bing_emotion_profile = {
    "QID":[],
    "rank": [],
    "IAS": [],
    "positive":[],
    "negative": [],
    "neutral": [],
    "anger":[],
    "anticipation":[],
    "disgust": [],
    "fear":[],
    "joy": [],
    "sadness": [],
    "surprise": [],
    "trust": [],
}

for _, row in tqdm(bing.iterrows(), total=len(bing)):
    bing_emotion_profile["QID"].append(row["QID"])
    resp = row["Resp"]
    bing_emotion_profile["rank"].append(row["Rank"])
    bing_emotion_profile["IAS"].append("Bing")
    emo_profile = calc_emotion_profile(resp)
    for key in emo_profile.keys():
        bing_emotion_profile[key].append(emo_profile[key])
    
bing_EP_df = pd.DataFrame.from_dict(bing_emotion_profile)
bing_EP_df.to_csv("../Results/LexIT_queryWise_bingEP.csv", index=False)
bing_EP_df.head()

  0%|          | 0/2925 [00:00<?, ?it/s]

,QID,rank,IAS,positive,negative,neutral,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,qGEN1,1,Bing,20.143376,22.574291,57.282332,0.0,14.285714,0.0,21.428571,21.428571,0.000000,28.571429,14.285714
1,qGEN1,2,Bing,17.607899,19.199351,63.192750,0.0,18.181818,0.0,36.363636,0.000000,18.181818,9.090909,18.181818
2,qGEN1,3,Bing,14.481309,17.352198,68.166492,12.5,0.000000,0.0,37.500000,0.000000,0.000000,25.000000,25.000000
3,qGEN1,4,Bing,16.979053,13.877154,69.143793,0.0,12.500000,0.0,25.000000,0.000000,0.000000,37.500000,25.000000
4,qGEN1,5,Bing,15.651291,17.619516,66.729192,0.0,0.000000,0.0,30.000000,0.000000,10.000000,40.000000,20.000000


In [ ]:
gemma = pd.read_csv("../Data/gemma_resp.csv")
gemma_emotion_profile = {
    "QID":[],
    "IAS":[],
    "positive":[],
    "negative": [],
    "neutral": [],
    "anger":[],
    "anticipation":[],
    "disgust": [],
    "fear":[],
    "joy": [],
    "sadness": [],
    "surprise": [],
    "trust": [],
}

for _, row in tqdm(gemma.iterrows(), total=len(gemma)):
    gemma_emotion_profile["QID"].append(row["QID"])
    gemma_emotion_profile["IAS"].append("Gemma")
    resp = row["Resp"]
    emo_profile = calc_emotion_profile(resp)
    for key in emo_profile.keys():
        gemma_emotion_profile[key].append(emo_profile[key])
    
gemma_EP_df = pd.DataFrame.from_dict(gemma_emotion_profile)
gemma_EP_df.to_csv("../Results/LexIT_queryWise_gemmaEP.csv", index=False)
gemma_EP_df.head()

  0%|          | 0/293 [00:00<?, ?it/s]

,QID,IAS,positive,negative,neutral,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,qGEN1,Gemma,24.469597,23.878635,51.651769,0.000000,0.000000,0.0,66.666667,0.000000,0.0,16.666667,16.666667
1,qGEN2,Gemma,24.469597,23.878635,51.651769,0.000000,0.000000,0.0,66.666667,0.000000,0.0,16.666667,16.666667
2,qGEN3,Gemma,17.429215,13.428776,69.142009,33.333333,33.333333,0.0,0.000000,33.333333,0.0,0.000000,0.000000
3,qGEN4,Gemma,16.400906,13.951157,69.647937,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
4,qGEN5,Gemma,19.006123,14.374663,66.619213,0.000000,66.666667,0.0,0.000000,0.000000,0.0,0.000000,33.333333


In [ ]:
gpt = pd.read_csv("../Data/gpt_resp.csv")

gpt_emotion_profile = {
    "QID":[],
    "IAS":[],
    "positive":[],
    "negative": [],
    "neutral": [],
    "anger":[],
    "anticipation":[],
    "disgust": [],
    "fear":[],
    "joy": [],
    "sadness": [],
    "surprise": [],
    "trust": [],
}

for _, row in tqdm(gpt.iterrows(), total=len(gpt)):
    gpt_emotion_profile["QID"].append(row["QID"])
    gpt_emotion_profile["IAS"].append("GPT")
    resp = row["Resp"]
    emo_profile = calc_emotion_profile(resp)
    for key in emo_profile.keys():
        gpt_emotion_profile[key].append(emo_profile[key])
    
gpt_EP_df = pd.DataFrame.from_dict(gpt_emotion_profile)
gpt_EP_df.to_csv("../Results/LexIT_queryWise_gptEP.csv", index=False)
gpt_EP_df.head()

  0%|          | 0/293 [00:00<?, ?it/s]

,QID,IAS,positive,negative,neutral,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,qGEN1,GPT,16.685771,17.050047,66.264182,1.666667,20.000000,1.666667,28.333333,1.666667,8.333333,16.666667,21.666667
1,qGEN2,GPT,16.014366,15.909519,68.076115,2.702703,8.108108,0.000000,36.486486,2.702703,5.405405,10.810811,33.783784
2,qGEN3,GPT,14.708838,11.832498,73.458664,5.172414,27.586207,1.724138,6.896552,12.068966,5.172414,3.448276,37.931034
3,qGEN4,GPT,14.868851,11.154594,73.976556,2.380952,23.809524,0.000000,4.761905,2.380952,4.761905,4.761905,57.142857
4,qGEN5,GPT,17.213599,15.847661,66.938740,7.142857,28.571429,7.142857,0.000000,7.142857,0.000000,7.142857,42.857143


### Language Model based approach: FEEL-IT

In [11]:
# santiy check

from transformers import pipeline
import pandas as pd
import re

emo_classifier = pipeline("text-classification",model='MilaNLProc/feel-it-italian-emotion',top_k=4)
sent_classifier = pipeline("text-classification",model='MilaNLProc/feel-it-italian-sentiment',top_k=2)

text_examples = [
    "sono felice",
    "La deforestazione è un male",
    "Questo è spaventoso",
    "Questa è una mela marcia",
    "mostro nero?",
    "lei è una bellezza terribile",
    "L'Cleanup Ocean Ã¨ un movimento internazionale che si Ã¨ formato per combattere la pollution del mare.",
    """La (tigre) 

    bianca Ã¨ una rara variante genetica della tigre reale (Panthera tigris), 
    
    caratterizzata da una particolare colorazione del mantello causata da una mutazione genetica. Questi bellissimi felini sono spesso oggetto di ammirazione e curiositÃ  per la loro singolare bellezza. Vive principalmente in India e in alcune parti del sud-est asiatico."""
]

def clean_text(text):
    text = re.sub("(\s?\n){1,}", ".", text) # remove new lines and replace with .
    text = re.sub("\t{1,}", " ", text) # remove tab spaces and replace with a singular space
    text = re.sub("[^\w\-(\.{1})'\!\?]", " ", text) # remove non-alphanumeric symbols, except for ., ', !, -
    text = re.sub("[\(\)\[\]\{\}]", "", text) # remove brackets of any kind
    text = re.sub("\s{2,}", " ", text) # remove any multiple white spaces
    text = text.strip() # remove any leading or ending white spaces
    return text

cleaned_text_examples = [clean_text(text) for text in text_examples]

emo_scores = emo_classifier(cleaned_text_examples)
sent_scores = sent_classifier(cleaned_text_examples)

EP_semantic_results = {
    "text": [],
    "cleaned_text": [],
    "positive": [],
    "negative": [],
    "sent_sum": [],
    "joy": [],
    "sadness": [],
    "anger": [],
    "fear": [],
    "emo_sum": []
}
# print(emo_scores)


for text, cleaned_text, sent_score, emo_score in zip(text_examples, cleaned_text_examples, sent_scores, emo_scores):
    EP_semantic_results["text"].append(text)
    EP_semantic_results["cleaned_text"].append(cleaned_text)
    sent_sum = 0
    emo_sum = 0
    for sent_dict in sent_score:
        EP_semantic_results[sent_dict["label"]].append(sent_dict["score"]*100)
        sent_sum += sent_dict["score"]
    EP_semantic_results["sent_sum"].append(sent_sum)
    for emo_dict in emo_score:
        EP_semantic_results[emo_dict["label"]].append(emo_dict["score"]*100)
        emo_sum += emo_dict["score"]
    EP_semantic_results["emo_sum"].append(emo_sum)

# print(EP_semantic_results)
df = pd.DataFrame.from_dict(EP_semantic_results)
df

Device set to use cpu
Device set to use cpu


,text,cleaned_text,positive,negative,sent_sum,joy,sadness,anger,fear,emo_sum
0,sono felice,sono felice,99.972600,0.027401,1.0,99.901080,0.041839,0.017075,0.040000,1.0
1,La deforestazione è un male,La deforestazione è un male,0.021573,99.978429,1.0,0.020934,92.929733,6.870111,0.179221,1.0
2,Questo è spaventoso,Questo è spaventoso,0.021750,99.978250,1.0,0.100756,0.102740,0.093424,99.703085,1.0
3,Questa è una mela marcia,Questa è una mela marcia,0.021880,99.978119,1.0,0.043510,70.564020,29.189795,0.202674,1.0
4,mostro nero?,mostro nero?,0.023057,99.976939,1.0,0.033823,3.354666,94.834334,1.777177,1.0
5,lei è una bellezza terribile,lei è una bellezza terribile,0.022565,99.977440,1.0,99.864703,0.107149,0.014439,0.013711,1.0
6,L'Cleanup Ocean Ã¨ un movimento internazionale...,L'Cleanup Ocean Ã un movimento internazionale ...,73.182118,26.817882,1.0,18.575227,48.435813,0.285598,32.703370,1.0
7,La (tigre) \n\n bianca Ã¨ una rara variante...,La tigre. bianca Ã una rara variante genetica ...,99.962687,0.037313,1.0,99.923551,0.029315,0.010889,0.036246,1.0


In [12]:
from transformers import pipeline
import pandas as pd
from tqdm.notebook import tqdm
import re

tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}
emo_classifier = pipeline("text-classification",model='MilaNLProc/feel-it-italian-emotion',top_k=4)
sent_classifier = pipeline("text-classification",model='MilaNLProc/feel-it-italian-sentiment',top_k=2)


def clean_text(text):
    text = re.sub("(\s?\n){1,}", ".", text) # remove new lines and replace with .
    text = re.sub("\t{1,}", " ", text) # remove tab spaces and replace with a singular space
    text = re.sub("[^\w\-(\.{1})'\!\?]", " ", text) # remove non-alphanumeric symbols, except for ., ', !, -
    text = re.sub("[\(\)\[\]\{\}]", "", text) # remove brackets of any kind
    text = re.sub("\s{2,}", " ", text) # remove any multiple white spaces
    text = text.strip() # remove any leading or ending white spaces
    return text

def get_semantic_EP(texts, text_name):
    cleaned_texts = [clean_text(text) for text in texts]
    cleaned_text_name = text_name+"_cleaned"
    try:
        emo_scores = emo_classifier(cleaned_texts)
        sent_scores = sent_classifier(cleaned_texts)
    except:
        emo_scores = emo_classifier(cleaned_texts, **tokenizer_kwargs)
        sent_scores = sent_classifier(cleaned_texts, **tokenizer_kwargs)

    EP_semantic_results = {
        cleaned_text_name: [],
        "positive": [],
        "negative": [],
        "joy": [],
        "sadness": [],
        "anger": [],
        "fear": [],
    }

    for cleaned_text, sent_score, emo_score in tqdm(zip(cleaned_texts, sent_scores, emo_scores), total=len(cleaned_texts)):
        EP_semantic_results[cleaned_text_name].append(cleaned_text)
        for sent_dict in sent_score:
            EP_semantic_results[sent_dict["label"]].append(sent_dict["score"]*100)
        for emo_dict in emo_score:
            EP_semantic_results[emo_dict["label"]].append(emo_dict["score"]*100)
    df = pd.DataFrame.from_dict(EP_semantic_results)
    return cleaned_text_name, df

Device set to use cpu
Device set to use cpu


In [ ]:
InsideOut = pd.read_csv("../Data/InsideOutData_compiled.csv")

col_name, IO_df = get_semantic_EP(InsideOut["Resp"].tolist(), "Resp")
IO_df["QID"] = InsideOut["QID"]
IO_df["rank"] = InsideOut["Rank"]
IO_df["Task Sentiment"] = InsideOut["Task Sentiment"]
IO_df["IAS"] = ["Bing"]*len(InsideOut)
IO_df = IO_df.loc[:, ["QID", "Task Sentiment", "IAS", "rank", col_name, "positive", "negative", "joy", "sadness", "anger", "fear"]]
IO_df.to_csv("../Results/FEELIT_queryWise_InsidOutEP.csv", index=False)
IO_df.head()

  0%|          | 0/1739 [00:00<?, ?it/s]

,QID,Task Sentiment,IAS,rank,Resp_cleaned,positive,negative,joy,sadness,anger,fear
0,qGEN1,NaN,Bing,1,Tornado - Wikipedia Tornado - Wikipedia Tornad...,99.909377,0.090624,11.563355,86.946112,0.490834,0.999694
1,qGEN2,NaN,Bing,2,Extreme up-close video of tornado near Wray CO...,99.975449,0.024549,99.937147,0.027813,0.013232,0.021811
2,qGEN3,NaN,Bing,3,Tornado facts and information. Tornadoes are v...,99.969923,0.030083,99.896598,0.048017,0.022982,0.032408
3,qGEN4,NaN,Bing,4,Tornado Central - weather.com. Tornado Season ...,0.666457,99.333537,2.168188,84.036607,0.040767,13.754435
4,qGEN5,NaN,Bing,5,Tornadoes Ready.gov. Tornadoes can destroy bui...,0.089091,99.910909,0.272096,0.116199,99.456948,0.154748


In [ ]:
query_file_name = ""
query_file_path = "../Data/"+query_file_name

queries = pd.read_csv(query_file_path)
col_name, queries_df = get_semantic_EP(queries["Query"].tolist(), "Query")
queries_df["QID"] = queries["QID"]
queries_df = queries_df.loc[:, ["QID", col_name, "positive", "negative", "joy", "sadness", "anger", "fear"]]
queries_df.to_csv("../Results/FEELIT_queryWise_queryEP.csv", index=False)
queries_df.head()

  0%|          | 0/293 [00:00<?, ?it/s]

,QID,Query_cleaned,positive,negative,joy,sadness,anger,fear
0,qGEN1,tornado,0.058053,99.941945,0.071032,99.501771,0.374920,0.052280
1,qGEN2,tornado,0.058053,99.941945,0.071032,99.501771,0.374920,0.052280
2,qGEN3,piramide egizia,0.035509,99.964488,18.792620,1.731412,11.087692,68.388271
3,qGEN4,piramidi,99.916172,0.083829,94.845456,2.796574,0.477019,1.880950
4,qGEN5,qual Ã la piramide egizia piÃ¹ alta,99.960083,0.039920,99.873513,0.039468,0.023573,0.063449


In [ ]:
gemma_resp = pd.read_csv("../Data/gemma_resp.csv")
col_name, gemma_df = get_semantic_EP(gemma_resp["Resp"].tolist(), "Resp")
gemma_df["QID"] = gemma_resp["QID"]
gemma_df["IAS"] = ["Gemma"]*len(gemma_resp)
gemma_df = gemma_df.loc[:, ["QID", "IAS", col_name, "positive", "negative", "joy", "sadness", "anger", "fear"]]
gemma_df.to_csv("../Results/FEELIT_queryWise_gemmaEP.csv", index=False)
gemma_df.head()

  0%|          | 0/293 [00:00<?, ?it/s]

,QID,IAS,Resp_cleaned,positive,negative,joy,sadness,anger,fear
0,qGEN1,Gemma,Il tornado è un'enorme tempesta atmosferica co...,18.078068,81.921935,83.850652,5.296346,0.019608,10.833394
1,qGEN2,Gemma,Il tornado è un'enorme tempesta atmosferica co...,18.078068,81.921935,83.850652,5.296346,0.019608,10.833394
2,qGEN3,Gemma,La piramide egizia è un monumento funerario di...,99.940920,0.059085,93.032485,2.686200,0.483063,3.798253
3,qGEN4,Gemma,Le piramidi sono strutture monumentali che si ...,99.890006,0.109995,99.629623,0.133649,0.037645,0.199084
4,qGEN5,Gemma,La piramide più alta dell'antico Egitto è la P...,99.952531,0.047469,99.833399,0.034575,0.032030,0.100000


In [ ]:
gpt_resp = pd.read_csv("../Data/gpt_resp.csv")
col_name, gpt_df = get_semantic_EP(gpt_resp["Resp"].tolist(), "Resp")
gpt_df["QID"] = gpt_resp["QID"]
gpt_df["IAS"] = ["GPT"]*len(gpt_resp)
gpt_df = gpt_df.loc[:, ["QID", "IAS", col_name, "positive", "negative", "joy", "sadness", "anger", "fear"]]
gpt_df.to_csv("../Results/FEELIT_queryWise_gptEP.csv", index=False)
gpt_df.head()

  0%|          | 0/293 [00:00<?, ?it/s]

,QID,IAS,Resp_cleaned,positive,negative,joy,sadness,anger,fear
0,qGEN1,GPT,Un tornado è un violento vortice d'aria in rot...,0.023756,99.976248,0.056199,0.318655,0.109930,99.515212
1,qGEN2,GPT,Il termine tornado si riferisce a un violento ...,0.074121,99.925882,0.098063,0.185901,0.073591,99.642450
2,qGEN3,GPT,Le piramidi egizie sono monumenti funerari cos...,99.842525,0.157480,99.892104,0.040411,0.013365,0.054116
3,qGEN4,GPT,Le piramidi sono strutture architettoniche di ...,99.958080,0.041924,99.918371,0.033479,0.011711,0.036430
4,qGEN5,GPT,La piramide egizia più alta è la Grande Pirami...,99.972552,0.027448,99.938476,0.025319,0.012664,0.023536


In [ ]:
bing_resp = pd.read_csv("../Data/bing_resp.csv")
col_name, bing_df = get_semantic_EP(bing_resp["Resp"].tolist(), "Resp")
bing_df["QID"] = bing_resp["QID"]
bing_df["rank"] = bing_resp["Rank"]
bing_df["IAS"] = ["Bing"]*len(bing_resp)
bing_df = bing_df.loc[:, ["QID", "IAS", "rank", col_name, "positive", "negative", "joy", "sadness", "anger", "fear"]]
bing_df.to_csv("../Results/FEELIT_queryWise_bingEP.csv", index=False)
bing_df.head()

  0%|          | 0/2925 [00:00<?, ?it/s]

,QID,IAS,rank,Resp_cleaned,positive,negative,joy,sadness,anger,fear
0,qGEN1,Bing,1,Tromba d'aria - Wikipedia. Una tromba d'aria o...,0.043484,99.956518,0.647116,5.918216,0.050854,93.383813
1,qGEN1,Bing,2,Cos'è un tornado? Come si forma? - Ilmeteo.net...,0.026361,99.973637,0.169292,0.259403,0.040697,99.530607
2,qGEN1,Bing,3,Che cosa sono e come nascono i tornado? - Focu...,0.029268,99.970728,0.216355,0.614346,0.035964,99.133331
3,qGEN1,Bing,4,Trombe d aria e tornado cosa sono come si svil...,0.219933,99.780065,98.734915,0.069436,0.023516,1.172134
4,qGEN1,Bing,5,Trombe d aria e tornado cosa sono perché nasco...,0.020265,99.979740,0.018684,94.916797,2.674790,2.389724
